<a href="https://colab.research.google.com/gist/ruvnet/433b24a201979e25051a4e772f883b21/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code-as-Action Agent using CrewAI and LangGraph

This notebook demonstrates a modular Code-as-Action agent implementation using CrewAI and LangGraph. The agent executes Python code based on user-specified tasks and includes a Gradio UI for interaction.

In [ ]:
# 1. Setup and Installation
!pip install crewai 'crewai[tools]' langgraph langchain langchain_openai openai gradio

In [ ]:
# 2. Import required libraries
from crewai import Agent, Task, Crew
from crewai_tools import CodeInterpreterTool
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import os
import io
import contextlib
import gradio as gr

In [ ]:
# Set up OpenAI API key and model name (replace YOUR_OPENAI_API_KEY with your actual key)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

In [ ]:
# 3. Agent Implementation

# Create a CrewAI agent with code execution enabled
coding_agent = Agent(
    role="Python Code Assistant",
    goal="Solve tasks by writing and executing Python code",
    backstory="An agent that uses Python coding skills to solve problems.",
    allow_code_execution=True,
    verbose=True
)

# Define a safe code execution function for sandboxing
def safe_execute_code(code: str) -> str:
    forbidden = ["import os", "import sys", "subprocess", "__", "open(", "eval("]
    if any(f in code for f in forbidden):
        return "Error: Disallowed code detected."
    safe_globals = {"__builtins__": {"len": len, "range": range}}
    output_buffer = io.StringIO()
    safe_locals = {"print": lambda *args, **kwargs: print(*args, **kwargs, file=output_buffer)}
    try:
        with contextlib.redirect_stdout(output_buffer):
            exec(code, safe_globals, safe_locals)
    except Exception as e:
        return f"Error during execution: {e}"
    result = output_buffer.getvalue()
    if result == "":
        result = "Execution finished (no output)."
    return result.strip()

# Create a LangGraph tool that executes code using the safe execution function
@tool
def run_code(code: str) -> str:
    return safe_execute_code(code)

# Initialize a LangGraph ReAct agent with the code execution tool
llm = ChatOpenAI(temperature=0)
code_agent_app = create_react_agent(model=llm, tools=[run_code])

In [ ]:
# Define a function to run tasks using either CrewAI or LangGraph agent
def run_agent(agent_framework: str, task_type: str, user_input: str):
    print(f"[LOG] Running task '{task_type}' using {agent_framework} agent...")
    try:
        if agent_framework == "CrewAI":
            if task_type == "Hello World":
                description = "Write and execute Python code that prints 'Hello World'."
            elif task_type == "Mathematical Computation":
                description = f"Calculate the result of the expression {user_input} using Python and output the result."
            elif task_type == "Text Processing":
                description = f"Write Python code to process the text: '{user_input}'. Specifically, reverse the string and output it."
            elif task_type == "API Simulation":
                description = ("Simulate an API request. Given input parameters, construct a JSON payload. "
                               f"For example, if input is '{user_input}', output a JSON object with that content.")
            else:
                description = user_input
            from crewai import Task, Crew
            task = Task(description=description, agent=coding_agent)
            crew = Crew(agents=[coding_agent], tasks=[task])
            result = crew.kickoff()
            output = str(result)
        else:  # LangGraph
            if task_type == "Hello World":
                prompt = "Print 'Hello World' using Python code."
            elif task_type == "Mathematical Computation":
                expr = user_input
                if "," in user_input:
                    parts = [p.strip() for p in user_input.split(",")]
                    if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                        expr = f"{parts[0]} + {parts[1]}"
                prompt = f"Calculate {expr} by writing and executing Python code."
            elif task_type == "Text Processing":
                prompt = f"Reverse the string '{user_input}' using Python code."
            elif task_type == "API Simulation":
                prompt = (f"Create a JSON object from the input '{user_input}' using Python (without actually calling an API). "
                          "Output the JSON structure.")
            else:
                prompt = user_input
            state = {"messages": [("user", prompt)]}
            final_state = code_agent_app.invoke(state)
            messages = final_state.get("messages", final_state)
            output = None
            if isinstance(messages, list) and len(messages) > 0:
                normalized_msgs = []
                for msg in messages:
                    if isinstance(msg, (tuple, list)):
                        role, content = msg[0], msg[1]
                    elif isinstance(msg, dict):
                        role, content = msg.get("role"), msg.get("content")
                    else:
                        continue
                    normalized_msgs.append((role, content))
                for role, content in reversed(normalized_msgs):
                    if role in ("assistant", "ai", "function"):
                        output = content
                        break
            if output is None:
                output = str(final_state)
        print(f"[LOG] Task result: {output}")
        return output
    except Exception as e:
        print(f"[ERROR] Agent execution failed: {e}")
        return f"Error: {e}"


In [ ]:
# 4. Example Use Cases: Run several tasks to demonstrate functionality
examples = [
    ("CrewAI", "Hello World", ""),
    ("CrewAI", "Mathematical Computation", "15, 27"),
    ("CrewAI", "Text Processing", "OpenAI"),
    ("CrewAI", "API Simulation", "temperature:25"),
    ("LangGraph", "Hello World", ""),
    ("LangGraph", "Mathematical Computation", "7, 8"),
    ("LangGraph", "Text Processing", "LangGraph"),
    ("LangGraph", "API Simulation", "status:ok")
]

for framework, task, inp in examples:
    print("="*50)
    print(f"Example: {framework} - {task} (input: '{inp}')")
    result = run_agent(framework, task, inp)
    print(f"Output: {result}\n")

In [ ]:
# 5. Create a Gradio UI for interaction
def handle_request(agent_choice, task_choice, user_input):
    return run_agent(agent_choice, task_choice, user_input or "")

with gr.Blocks() as demo:
    gr.Markdown("## 🚀 Code-as-Action Agent Demo")
    gr.Markdown("Select an agent framework and a task type, provide input if needed, and run the agent.")
    agent_select = gr.Dropdown(["CrewAI", "LangGraph"], label="Agent Framework", value="CrewAI")
    task_select = gr.Dropdown(
        ["Hello World", "Mathematical Computation", "Text Processing", "API Simulation"],
        label="Task Type",
        value="Hello World"
    )
    user_input_box = gr.Textbox(label="Input (if required)", placeholder="E.g., for Math: '3,4', for Text: 'hello', for API: 'key:value'")
    output_box = gr.Textbox(label="Agent Output")
    run_btn = gr.Button("Run Agent")
    run_btn.click(fn=handle_request, inputs=[agent_select, task_select, user_input_box], outputs=output_box)

demo.launch(share=False)